In [1]:
import yahooquery as yf
import locale
import pandas as pd

locale.setlocale(locale.LC_ALL, 'pt_BR')

'pt_BR'

In [14]:
simbolos = [
'VALE3.SA',
'ITUB4.SA',
'PETR4.SA',
'BBDC4.SA',
'B3SA3.SA',
'PETR3.SA',
'ELET3.SA',
'ABEV3.SA',
'RENT3.SA',
'WEGE3.SA',
'BBAS3.SA',
'ITSA4.SA',
'SUZB3.SA',
'BPAC11.SA',
'HAPV3.SA',
'EQTL3.SA',
'RADL3.SA',
'RDOR3.SA',
'PRIO3.SA',
'LREN3.SA',
'GGBR4.SA',
'JBSS3.SA',
'RAIL3.SA',
'BBDC3.SA',
'VBBR3.SA',
'ENEV3.SA',
'CSAN3.SA',
'SBSP3.SA',
'BBSE3.SA',
'HYPE3.SA',
'VIVT3.SA',
'CMIG4.SA',
'TOTS3.SA',
'ASAI3.SA',
'UGPA3.SA',
'KLBN11.SA',
'ELET6.SA',
'CCRO3.SA',
'MGLU3.SA',
'BRFS3.SA',
'CPLE6.SA',
'ENGI11.SA',
'NTCO3.SA',
'TIMS3.SA',
'SANB11.SA',
'EGIE3.SA',
'CRFB3.SA',
'EMBR3.SA',
'AMER3.SA',
'RRRP3.SA',
'BRML3.SA',
'TAEE11.SA',
'BRKM5.SA',
'GOAU4.SA',
'CSNA3.SA',
'SOMA3.SA',
'MULT3.SA',
'SULA11.SA',
'FLRY3.SA',
'CPFE3.SA',
'ARZZ3.SA',
'CIEL3.SA',
'BRAP4.SA',
'COGN3.SA',
'RAIZ4.SA',
'ENBR3.SA',
'AZUL4.SA',
'VIIA3.SA',
'CYRE3.SA',
'IGTI11.SA',
'YDUQ3.SA',
'LWSA3.SA',
'SLCE3.SA',
'ALPA4.SA',
'SMTO3.SA',
'CMIN3.SA',
'USIM5.SA',
'BEEF3.SA',
'MRFG3.SA',
'PCAR3.SA',
'MRVE3.SA',
'PETZ3.SA',
'DXCO3.SA',
'BPAN4.SA',
'IRBR3.SA',
'QUAL3.SA',
'CVCB3.SA',
'ECOR3.SA',
'EZTC3.SA',
'GOLL4.SA',
'POSI3.SA',
'CASH3.SA']

In [83]:
def generateData(simbol):
    tkr = yf.Ticker(simbol)
    tkr_all = tkr.all_modules[simbol]
    
    try:
        balance = tkr_all['balanceSheetHistoryQuarterly']['balanceSheetStatements'][0]
    except:
        balance = tkr_all['balanceSheetHistory']['balanceSheetStatements'][0]
    
    #Total Enterprise Value

    marketCap = tkr_all['summaryDetail']['marketCap']
    
    try:
        #TEV = marketCap + balance['totalCurrentLiabilities'] - balance['totalCurrentAssets']
        #TEV = marketCap + balance['shortLongTermDebt'] + balance['longTermDebt'] - balance['cash']
        TEV = marketCap + tkr_all['financialData']['totalDebt'] - tkr_all['financialData']['totalCash']
    except:
        TEV = 0
    
    #Earning Yield

    ebit = tkr_all['incomeStatementHistoryQuarterly']['incomeStatementHistory'][0]['ebit']
    
    if not ebit > 1 or TEV == 0:
        return None

    EY = ebit / TEV
    EY = round(EY*100, 2)
    
    
    #invested Capital
    try:
        shortLongTermDebt = balance['shortLongTermDebt']
    except:
        shortLongTermDebt = 0
    try:
        longTermDebt = balance['longTermDebt']
    except:
        longTermDebt = 0
    try:
        totalStockholderEquity = balance['totalStockholderEquity']
    except:
        totalStockholderEquity = 0
    try:
        goodWill = balance['goodWill']
    except:
        goodWill = 0
    try:
        cash = balance['cash']
    except:
        cash = 0
    try:
        retainedEarnings = balance['retainedEarnings']
    except:
        retainedEarnings = 0

    invCap = shortLongTermDebt + longTermDebt + totalStockholderEquity + goodWill + cash + retainedEarnings
    
    # Return on Invested Capital
    if ebit > 0:
        ROIC = ebit / invCap
        ROIC = round(ROIC*100, 2)
    else:
        ROIC = 0
    
    # magic index
    MAGIC_IDX = round(EY + ROIC, 2)
    
    try:
        DY = round(tkr_all['summaryDetail']['dividendYield']*100, 2)
    except:
        DY = 0
        
    recommendationTrend = tkr_all['recommendationTrend']['trend'][3]
    buy = recommendationTrend['strongBuy'] + recommendationTrend['buy']
    sell = recommendationTrend['strongSell'] + recommendationTrend['sell']
    
    data = {
        'Empresa': tkr_all['quoteType']['longName'],
        'Setor': tkr_all['summaryProfile']['sector'],
        'Ticker': simbol,
        'MagicIndex': MAGIC_IDX,
        'EarningYield': EY,
        'ROIC': ROIC,
        'DividendYield': DY ,
        'Price': tkr_all['financialData']['currentPrice'],
        'Recomendacao Compra': buy,
        'Recomendacao Venda': sell,
        'MarketCap': marketCap
        
    }
    
    return data
    

In [84]:
all_data = []
for i in simbolos:
    try:
        d = generateData(i)
        if d:
            all_data.append(d)
    except:
        print('Nao rodou para ', i)
        print('---------------')
        print('---------------')
        pass

Nao rodou para  RAIZ4.SA
---------------
---------------
Nao rodou para  IGTI11.SA
---------------
---------------


In [85]:
df = pd.DataFrame(all_data)

In [86]:
df.sort_values(by='MagicIndex', ascending=False, ignore_index=True)

,Empresa,Setor,Ticker,MagicIndex,EarningYield,ROIC,DividendYield,Price,Recomendacao Compra,Recomendacao Venda,MarketCap
0,Metalurgica Gerdau S.A.,Basic Materials,GOAU4.SA,36.16,25.90,10.26,12.73,11.75,4,0,15708928000
1,Petróleo Brasileiro S.A. - Petrobras,Energy,PETR4.SA,29.03,17.21,11.82,34.99,28.30,8,2,393907666944
2,Petróleo Brasileiro S.A. - Petrobras,Energy,PETR3.SA,29.00,17.18,11.82,31.31,31.71,4,2,394948050944
3,Marfrig Global Foods S.A.,Consumer Defensive,MRFG3.SA,17.96,11.12,6.84,18.24,11.53,4,1,7606167552
4,Gerdau S.A.,Basic Materials,GGBR4.SA,16.72,11.37,5.35,11.03,26.96,10,1,41866448896
...,...,...,...,...,...,...,...,...,...,...,...
68,Locaweb Serviços de Internet S.A.,Technology,LWSA3.SA,0.85,0.49,0.36,0.00,10.08,0,0,5941776896
69,Itaúsa S.A.,Financial Services,ITSA4.SA,0.74,0.37,0.37,0.88,10.77,5,0,95051718656
70,Natura &Co Holding S.A.,Consumer Defensive,NTCO3.SA,0.05,0.03,0.02,0.88,14.35,0,0,19704989696
71,Azul S.A.,Industrials,AZUL4.SA,0.02,0.50,-0.48,0.00,16.14,5,0,5615250944


In [87]:
import datetime

fileName = f'magic_formula2_{datetime.datetime.now().strftime("%d%m%Y-%H%M%S")}.xlsx'

df.to_excel(fileName, index = False)